In [1]:
!wget https://zenodo.org/record/3406662/files/test_labels.txt.gz?download?=1 -O test_labels.txt.gz
!wget https://zenodo.org/record/3406662/files/valid_labels.txt.gz?download?=1 -O valid_labels.txt.gz
!wget https://zenodo.org/record/3406662/files/testpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz?download=1 -O testpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
!wget https://zenodo.org/record/3406662/files/validpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz?download=1 -O validpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz

--2020-11-20 17:23:44--  https://zenodo.org/record/3406662/files/test_labels.txt.gz?download?=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6001 (5.9K) [application/octet-stream]
Saving to: ‘test_labels.txt.gz’

test_labels.txt.gz  100%[===================>]   5.86K  --.-KB/s    in 0s      

2020-11-20 17:23:45 (498 MB/s) - ‘test_labels.txt.gz’ saved [6001/6001]

--2020-11-20 17:23:45--  https://zenodo.org/record/3406662/files/valid_labels.txt.gz?download?=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5969 (5.8K) [application/octet-stream]
Saving to: ‘valid_labels.txt.gz’

valid_labels.txt.gz 100%[===================>]   5.83K  --.-KB/s    in 0s      

2020-11-20 17:23:46 (531 MB/s) - ‘valid_labels.txt.gz’ saved [5969/5969]


In [2]:
import gzip
import glob
import numpy as np
from collections import defaultdict
from abstention.calibration import softmax, TempScaling
from abstention.label_shift import EMImbalanceAdapter

In [3]:
def read_labels(fh):
    to_return = []
    for line in fh:
        the_class=int(line.rstrip())
        to_add = np.zeros(10)
        to_add[the_class] = 1
        to_return.append(to_add)
    return np.array(to_return)

test_labels = read_labels(gzip.open(glob.glob("test_labels.txt.gz")[0]))
valid_labels = read_labels(gzip.open(glob.glob("valid_labels.txt.gz")[0]))

def read_preds(fh):
    return np.array([[float(x) for x in y.decode("utf-8").rstrip().split("\t")]
                     for y in fh])

test_preds = softmax(preact=read_preds(gzip.open(glob.glob("testpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz")[0])),
                     temp=1, biases=None)
valid_preds = softmax(preact=read_preds(gzip.open(glob.glob("validpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz")[0])),
                      temp=1, biases=None)

In [4]:
def sample_from_probs_arr(arr_with_probs):
    rand_num = np.random.random()
    cdf_so_far = 0
    for (idx, prob) in enumerate(arr_with_probs):
        cdf_so_far += prob
        if (cdf_so_far >= rand_num
            or idx == (len(arr_with_probs) - 1)):  # need the
            # letterIdx==(len(row)-1) clause because of potential floating point errors
            # that mean arrWithProbs doesn't sum to 1
            return idx
        
test_class_to_indices = defaultdict(list)
for index,row in enumerate(test_labels):
    row_label = np.argmax(row)
    test_class_to_indices[row_label].append(index)

def draw_test_indices(total_to_return, label_proportions):
    indices_to_use = []
    for class_index, class_proportion in enumerate(label_proportions):
        indices_to_use.extend(np.random.choice(
                test_class_to_indices[class_index],
                int(total_to_return*class_proportion),
                replace=True))
    for i in range(total_to_return-len(indices_to_use)):
        class_index = sample_from_probs_arr(label_proportions)
        indices_to_use.append(
            np.random.choice(test_class_to_indices[class_index]))
    return indices_to_use

dirichlet_alpha = 0.1
samplesize = 1000
dirichlet_dist = np.random.dirichlet([dirichlet_alpha for x in range(10)])
test_indices = draw_test_indices(total_to_return=samplesize,
                                 label_proportions=dirichlet_dist)
shifted_test_labels = test_labels[test_indices]
shifted_test_preds = test_preds[test_indices]

In [5]:
imbalance_adapter = EMImbalanceAdapter(calibrator_factory =
        TempScaling(verbose=False, bias_positions='all'))

imbalance_adapter_func = imbalance_adapter(valid_labels=valid_labels,
                      tofit_initial_posterior_probs=shifted_test_preds,
                      valid_posterior_probs=valid_preds)

adapted_shifted_test_preds = imbalance_adapter_func(shifted_test_preds)

In [6]:
test_accuracy = np.mean(np.argmax(shifted_test_labels,axis=-1)==np.argmax(shifted_test_preds,axis=-1))
adapted_test_accuracy = np.mean(np.argmax(shifted_test_labels,axis=-1)==np.argmax(adapted_shifted_test_preds,axis=-1))
print(test_accuracy, adapted_test_accuracy)

0.903 0.961
